In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import requests

**Local:**

In [3]:
import constants
import constraints

In [4]:
constants.pandas_settings()

In [9]:
dg_url='https://feeds.datagolf.com/field-updates?tour=pga&file_format=json&key=6a626b6c312c0d33cfe157d614b5'
res = requests.get(dg_url) 

ValueError: Invalid file path or buffer object type: <class 'dict'>

In [5]:
def edit_fanduel():
    df = pd.read_csv(f'../data/contest-files/{constants.tournament}.csv', usecols=constants.keep_cols)
    df.columns = df.columns.str.lower()

    ret = (df
           .rename({'nickname': 'name'}, axis=1)
           #.loc[(df['injury indicator']!='O') & (df['salary']>7000)]
           .drop('injury indicator', axis=1)
           .dropna()
           .reset_index(drop=True)
          )
    
    col_types = { 'name': 'str', 'fppg': 'float' }
    
    for col in ret.columns:
        ret[col] = ret[col].astype(col_types.get(col,'int'))
    
    ret.to_pickle('../data/pickle-buffer/fanduel-data.pkl')
    
    return None


def load_fanduel():
    
    edit_fanduel()
    
    return pd.read_pickle('../data/pickle-buffer/fanduel-data.pkl')

In [9]:
edit_fanduel()

             nickname  fppg  played  salary injury indicator
0        Rory McIlroy 78.01   13.00   12000              NaN
1   Scottie Scheffler 80.40   20.00   11900              NaN
2            Jon Rahm 73.35   14.00   11800              NaN
3       Justin Thomas 84.55   17.00   11700              NaN
4         Shane Lowry 61.60   12.00   11500              NaN
5     Collin Morikawa 68.14   14.00   11400              NaN
6   Xander Schauffele 77.94   16.00   11300              NaN
7    Matt Fitzpatrick 63.88   16.00   11200              NaN
8       Jordan Spieth 64.42   18.00   11100              NaN
9      Will Zalatoris 62.94   17.00   11000              NaN
10      Cameron Smith 72.89   14.00   10900              NaN
11     Dustin Johnson 60.67   10.00   10800              NaN
12    Patrick Cantlay 76.78   13.00   10700              NaN
13     Viktor Hovland 71.50   16.00   10600              NaN
14   Hideki Matsuyama 72.27   17.00   10500              NaN


In [6]:
strokes_gained_components = {
    'tee': {
        'url_id': 2567,
        #'url': 'https://www.pgatour.com/stats/stat.02567.html',
        'shortened': 'ott'
    },
    'approach': {
        'url_id': 2568,
        #'url': 'https://www.pgatour.com/stats/stat.02568.html',
        'shortened': 'app'
    },
    'around': {
        'url_id': 2569,
        #'url': 'https://www.pgatour.com/stats/stat.02569.html',
        'shortened': 'arg'
    },
    'green': {
        'url_id': 2564,
        #'url': 'https://www.pgatour.com/stats/stat.02564.html',
        'shortened': 'putt'
    },
    'tee-to-green': {
        'url_id': 2674,
        #'url': 'https://www.pgatour.com/stats/stat.02674.html',
        'shortened' : 'ttg'
        }
}

new_col_names = {
    'player name': 'name',
    'rank this week': ' cur-rank',
    'rank last week': ' prev-rank',
    'average': ' sg',
    'rounds': ' num-rounds',
    'measured rounds': ' num-measured'
}

abbrev_col_names = [ 'name', ' sg' ]

In [7]:
def strokes_gained_per(golf_shot, abbreviate=True):
    
    if golf_shot.lower() not in strokes_gained_components:
        return None
    
    else:
        info = strokes_gained_components.get(golf_shot.lower(), None)
        if info is None:
            return None
        
        else:
            url = f'https://www.pgatour.com/stats/stat.0{ info["url_id"] }.html'
            
            ret = pd.read_html(url)[1].reset_index(drop=True)
            
            ret.columns = ret.columns.str.lower().str.replace('total sg:', ' sg').str.replace('\xa0', ' ')
            
            ret = ret.rename(new_col_names, axis=1)
            # ret.index = ret['name']
            # ret = ret.drop('name', axis=1)
            if abbreviate:
                ret = ret.loc[:, abbrev_col_names]
            
            ret.columns = ret.columns.str.replace(' ', f'{strokes_gained_components[golf_shot]["shortened"]}-')
            
            ret.to_pickle(f'../data/pickle-buffer/{strokes_gained_components[golf_shot]["shortened"]}-sg.pkl')
            
            return None
        
def load_strokes_gained_per(golf_shot):
    
    #**
    strokes_gained_per(golf_shot)
    
    return pd.read_pickle(f'../data/pickle-buffer/{strokes_gained_components[golf_shot]["shortened"]}-sg.pkl')

In [8]:
def aggregate_strokes_gained():
    
    # Create dictionary containing strokes-gained data for each stroke
    sg_frames = { golf_shot: load_strokes_gained_per(golf_shot) for golf_shot in strokes_gained_components }

    # Initialize frame as tee and merge rest of shots
    sgdf = sg_frames['tee']
    for k in list(strokes_gained_components.keys())[1:]:
        sgdf = sgdf.merge(sg_frames[k])
    
    ret = (sgdf
           .sort_values(by=constants.focus_stat, ascending=False)
           .reset_index(drop=True)
          )

    ret.to_pickle(f'../data/pickle-buffer/strokes-gained.pkl')                                                 
                                                 
    return None

def load_strokes_gained():
    
    aggregate_strokes_gained()
    
    return pd.read_pickle('../data/pickle-buffer/strokes-gained.pkl')

In [ ]:
def combine_pga_fanduel():
    fd = load_fanduel()
    sg = load_strokes_gained()
    
    focus_stats = [ constants.focus_stat ]
    if constants.focus_stat_2 is not None:
        focus_stats.append(constants.focus_stat_2)
        if constants.focus_stat_3 is not None:
            focus_stats.append(constants.focus_stat_3)
    
    focus_stats = tuple(focus_stats)
    
    for sg_col in focus_stats:
        fd[sg_col] = fd['name'].apply(lambda x: sg.loc[x, sg_col] if x in sg.index else 0.0)
        fd[f'{sg_col}-per-10k'] = np.array( 10000 * fd[sg_col] / fd['salary'] )
    
    #convs = {'name': 'str', 'salary': 'int'}
    
    #for col in fd.columns:
        #fd[col] = fd[col].astype(convs.get(col, 'float'))
    
    # fd.index = fd['name']
    # fd = fd.drop('name', axis=1)
                           
    fd = (fd
          .sort_values(by=[constants.focus_stat], ascending=False)
          .dropna()
         )
    
    fd.to_pickle(f'../data/pickle-buffer/{constants.tournament}.pkl')

    return None

In [ ]:
import itertools
from itertools import combinations

from functools import cache
from tqdm.notebook import tqdm

#from pandarallel import pandarallel
#pandarallel.initialize(use_memory_fs=True)

In [ ]:
def add_constraints():
    
    combine_pga_fanduel()
    ret = pd.read_pickle(f'../data/pickle-buffer/{constants.tournament}.pkl')
    
    if constraints.min_salary is not None:
        
        ret = (ret
               .loc[ ret['salary']>=constraints.min_salary ]
               .reset_index(drop=True)
              )
    
    ret.to_pickle(f'../data/pickle-buffer/optimizer-data.pkl')
    
    return None

In [ ]:
def prepare_input():
    
    add_constraints()
    ret = pd.read_pickle(f'../data/pickle-buffer/optimizer-data.pkl')
    ret_names = ret['name'].values.tolist()
    
    ret['salary'] /= 100
    ret.index = ret['name']
    ret = ret.drop('name', axis=1)
    
    ret.to_pickle(f'../data/pickle-buffer/optimizer-data-clean.pkl')
    
    return ret_names
    

In [ ]:
pnames = prepare_input()
data = pd.read_pickle(f'../data/pickle-buffer/optimizer-data-clean.pkl')

@cache
def get_value(name, column):
    return( data.loc[name, column] )

@cache
def sum_values(names, column):
    return( sum( [ get_value(name, column) for name in names ] ) )

@cache
def is_valid_lineup(lineup):
    return( sum_values(lineup, 'salary') in constraints.cost_range and len(set(lineup))==6 )

@cache
def lineup_analysis(lineup):
    return(tuple( [ sum_values(tuple(set(lineup)),column) for column in constraints.cols_to_sum ] ) )

def lineup_analysis_wrapper(lineup):
    return( lineup_analysis(tuple(set(lineup.to_numpy()))) if is_valid_lineup(tuple(set(lineup.to_numpy()))) else (0.0,)*len(constraints.cols_to_sum)  )

def create_lineup_2_slices(slate_dict):
#     2 things of three
    ret_list = list()
    
    for half_slates in tqdm( [p for p in itertools.product(*slate_dict.values())] ):
        
        g1,g2,g3 = tuple(sorted(list(half_slates[0])))
        g4,g5,g6 = tuple(sorted(list(half_slates[1])))
        
        lu = (g1,g2,g3,g4,g5,g6)
        if is_valid_lineup(lu):
            ret_list.append(lu)
    
    return(tuple(ret_list))

def create_lineup_3_slices(slate_dict):
    
#     3 things of two
    return None

# Trying to get better about only passing tuples or other completely immutable for default and for cache
def create_lineups():
    
    # Not necessary but makes reading easier
    num_players = 6 # (n)
    num_slices = constraints.slices
    step = int( len(pnames) / num_slices )
    
    r = int(num_players / num_slices) # (nCr)
    

    # slates = {f'slate{i+1}': tuple(map( tuple, itertools.combinations(pnames[:int(i*step)], r) )) for i in range(num_slices+1)}
    # slates.update( { f'slate{num_slices}': tuple(map( tuple, itertools.combinations(pnames[int(num_slices*step):], r) )) } )
    
    slates = dict()
    
    if num_slices == 2:
        
        slates = {
            'slate1': tuple(map( tuple, itertools.combinations(pnames[:step], r) )),
            'slate2': tuple(map( tuple, itertools.combinations(pnames[step:], r) ))
        }
        
    elif num_slices == 3:
        
        slates = {
            'slate1': tuple(map( tuple, itertools.combinations(pnames[:step], r) )),
            'slate2': tuple(map( tuple, itertools.combinations(pnames[step:int(2*step)], r) )),
            'slate3': tuple(map( tuple, itertools.combinations(pnames[int(2*step):], r) )),
            
        }
    
    operations = { 2: create_lineup_2_slices(slates), 3: create_lineup_3_slices(slates)}
    
    lineups = operations[num_slices]
    
    ret = pd.DataFrame(lineups, columns=['g1','g2','g3','g4','g5','g6'])
    #ret[constraints.cols_to_sum] = ret.apply( lineup_analysis_wrapper, axis=1, result_type='expand')
    ret[constraints.cols_to_sum] = ret.parallel_apply( lineup_analysis_wrapper, axis=1, result_type='expand' )
    
    ret['total-sg'] = ret.loc[:, constraints.sg_cols].sum(axis=1)
    ret['avg-value'] = ret.loc[:, constraints.val_cols].mean(axis=1)
    
    ret = (ret
           # .sort_values(by=f'{constants.focus_stat}-per-10k', ascending=False)
           .sort_values(by='avg-value', ascending=False)
           .drop_duplicates()
           .reset_index(drop=True)
          )
    
    
    ret.to_pickle(f'../data/lineups-created/{constants.tournament_fname()}.pkl')
    
    print('Done...')
    
    return None

def output_lineups(top_num=100):
    return pd.read_pickle(f'../data/lineups-created/{constants.tournament_fname()}.pkl').head(top_num)
    

In [ ]:
def create_func():
    if constants.create:
        create_lineups()
    return output_lineups()

In [ ]:
create_func()